In [1]:
from dask.distributed import Client, LocalCluster

In [2]:
client = Client(LocalCluster())

/Users/dkhosla/miniconda2/envs/dask-test/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
client

Client Scheduler: tcp://127.0.0.1:57034 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 17.18 GB


In [4]:
import dask.dataframe as dd

In [20]:
!ls | grep csv | xargs wc -l

 9710126 yellow_tripdata_2017-01.csv
 9168827 yellow_tripdata_2017-02.csv
 10294630 yellow_tripdata_2017-03.csv
 10046190 yellow_tripdata_2017-04.csv
 10102126 yellow_tripdata_2017-05.csv
 9656995 yellow_tripdata_2017-06.csv
 8539479 yellow_tripdata_2017-07.csv
 67518373 total


In [11]:
df_nyc = dd.read_csv("yellow_tripdata_2017-*.csv")

In [14]:
df_nyc.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2017-01-09 11:13:28,2017-01-09 11:25:45,1,3.3,1,N,263,161,1,12.5,0.0,0.5,2.00,0.0,0.3,15.30
1,1,2017-01-09 11:32:27,2017-01-09 11:36:01,1,0.9,1,N,186,234,1,5.0,0.0,0.5,1.45,0.0,0.3,7.25


In [17]:
df_nyc.payment_type.value_counts().head()

1    40646179
2    19382463
3      315223
4       91611
5           3
Name: payment_type, dtype: int64

In [25]:
len(df_nyc)

60435479

In [ ]:
(df_nyc
 .groupby('passenger_count')
 .apply(lambda gr: gr['trip_distance'].mean().compute())
)

## Writing to Parquet

In [ ]:
df = df_nyc.astype({'VendorID': 'uint8',
                    'passenger_count': 'uint8',
                    'RateCodeID': 'uint8',
                    'payment_type': 'uint8'})

df.to_parquet('s3://dask-data/nyc-taxi/tmp/parquet',
              compression='snappy',
              has_nulls=False,
              object_encoding='utf8',
              fixed_text={'store_and_fwd_flag': 1})